# High Accuracy CNN for MNIST

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [6]:
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist

X_train_full = X_train_full / 255.
X_test = X_test / 255.

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [7]:
np.random.seed(42)
tf.random.set_seed(42)
tf.keras.backend.clear_session()

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=3, padding="same",
                           activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Conv2D(64, kernel_size=3, padding="same",
                           activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [10]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 35s 20ms/step - accuracy: 0.8874 - loss: 0.3787 - val_accuracy: 0.9892 - val_loss: 0.0435
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 33s 19ms/step - accuracy: 0.9735 - loss: 0.0887 - val_accuracy: 0.9890 - val_loss: 0.0462
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 33s 19ms/step - accuracy: 0.9808 - loss: 0.0642 - val_accuracy: 0.9902 - val_loss: 0.0359
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 32s 19ms/step - accuracy: 0.9849 - loss: 0.0485 - val_accuracy: 0.9914 - val_loss: 0.0358
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 33s 19ms/step - accuracy: 0.9870 - loss: 0.0389 - val_accuracy: 0.9908 - val_loss: 0.0409
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 32s 19ms/step - accuracy: 0.9882 - loss: 0.0360 - val_accuracy: 0.9920 - val_loss: 0.0380
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 33s 19ms/step - accuracy: 0.9900 - loss: 0.0308 - val_accuracy: 0.9916 - val_loss: 0.0427
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.9907 -

[0.035846397280693054, 0.9921000003814697]

In [12]:
testData = pd.read_csv("test.csv")

xTest = testData.values.astype("float32") / 255.0
xTest = xTest.reshape(-1, 28, 28, 1)

predProbabilities = model.predict(xTest, batch_size=128, verbose=1)
predLabels = np.argmax(predProbabilities, axis=1)

submission = pd.DataFrame({
    "ImageId": np.arange(1, len(predLabels) + 1),
    "Label":    predLabels
})

submission.to_csv("submission.csv", index=False)
print("Wrote submission.csv with", len(submission), "entries.")

219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step
Wrote submission.csv with 28000 entries.
